In [1]:
from typing import Annotated
from dotenv import load_dotenv
from pydantic import BaseModel
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
import gradio as gr


In [2]:
load_dotenv(override=True)

True

In [3]:
def add_value(left, right):
    if right is not None:
        return right
    return left

class State(BaseModel):
    email: Annotated[str, add_value] = ""
    is_valid: Annotated[bool, add_value] = False


In [4]:
graph_builder = StateGraph(State)

In [5]:
checkpointer = MemorySaver()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [6]:
# Node that asks GPT-4o to validate the email and return a boolean
# Returns dict to merge into state

def validate_email(state: State):
    prompt = (
        "You are an email validator. Reply with only 'true' or 'false'. "
        "If the email is valid RFC 5322 style and realistic, reply true; otherwise false. "
        f"Email: {state.email}"
    )
    resp = llm.invoke(prompt)
    decision = str(resp.content).strip().lower()
    is_valid = decision.startswith("t")
    return {"email": state.email, "is_valid": is_valid}

In [7]:
graph_builder.add_node("validate_email", validate_email)

In [8]:
# Wire the graph: START -> validate_email -> END
graph_builder.add_edge(START, "validate_email")
graph_builder.add_edge("validate_email", END)

graph = graph_builder.compile(checkpointer=checkpointer)

In [9]:
config = {"configurable": {"thread_id": "1"}}

In [10]:
def submit_email(user_email: str):
    initial_state = State(email=user_email)
    result_state = graph.invoke(initial_state, config=config)
    if result_state["is_valid"]:
        return f"อีเมลถูกบันทึกแล้ว: {result_state['email']}"
    return "อีเมลไม่ถูกต้อง กรุณากรอกใหม่"

In [11]:


# Simple UI for collecting the email from the user
gr.Interface(
    fn=submit_email,
    inputs=gr.Textbox(label="กรอกอีเมล"),
    outputs=gr.Textbox(label="ผลลัพธ์"),
    title="LangGraph Email Capture",
    description="ใส่อีเมลเพื่อบันทึกลง State ของ LangGraph",
).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio\flagged\dataset1.csv
